In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn import preprocessing
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

# Pre Processing

In [2]:
def extract_x(file):
    
    df = pd.read_csv(file)
    
    if file == 'epl.csv':
        x = df.drop(['fixture_date', 'fixture_id', 'Home Team', 'Away Team', 'Full Time Result',
                    'Result No Draw', 'A Win', 'B Win'], axis=1)
        x = pd.get_dummies(x, columns=["Promoted"], prefix="Promoted")

    elif file == 'nba.csv':
        x = df.drop([ 'Home Win', 'MATCH_DATE', 'HOME_TEAM', 'AWAY_TEAM', 
                                        'A Win', 'B Win'], axis=1)
        
    else:
        x = df.drop(['FIGHTER_A', 'FIGHTER_B', 'Result', 'A Win', 'B Win', 'A_CUMULATIVE_RDS', 'B_CUMULATIVE_RDS'], axis=1)
    
    return x

In [3]:
def extract_y(file):
    
    df = pd.read_csv(file)
    
    if file == 'epl.csv':
        y = df['Result No Draw']
    elif file == 'nba.csv':
        y = df['Home Win']
    else:
        y = df['Result']
    
    return y

In [4]:
def extract_odds(file):
    
    df = pd.read_csv(file)
    odds = df[['A Win', 'B Win']]
    
    return odds

In [5]:
epl_x = extract_x('epl.csv')
epl_y = extract_y('epl.csv')
epl_odds = extract_odds('epl.csv')

nba_x = extract_x('nba.csv')
nba_y = extract_y('nba.csv')
nba_odds = extract_odds('nba.csv')

ufc_x = extract_x('ufc.csv')
ufc_y = extract_y('ufc.csv')
ufc_odds = extract_odds('ufc.csv')

# Static Variables

In [6]:
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1_score': make_scorer(f1_score),
           'auc': make_scorer(roc_auc_score)}

In [7]:
logreg = LogisticRegression(max_iter=10000)
knn = KNeighborsClassifier(n_neighbors=5)
forest = RandomForestClassifier()
svm = CalibratedClassifierCV(LinearSVC(dual=False))

# Cross Validation

In [8]:
def cross_validation_results(model):
    
    epl = cross_validate(model, epl_x, epl_y, cv=5, scoring=scoring)
    nba = cross_validate(model, nba_x, nba_y, cv=5, scoring=scoring)
    ufc = cross_validate(model, ufc_x, ufc_y, cv=5, scoring=scoring)
    
    epl_results = pd.DataFrame(epl)
    nba_results = pd.DataFrame(nba)
    ufc_results = pd.DataFrame(ufc)
    
    epl_results = epl_results.describe()
    nba_results = nba_results.describe()
    ufc_results = ufc_results.describe()
    
    combined = epl_results.join(nba_results, lsuffix='_epl', rsuffix='_nba').join(ufc_results)
    
    return combined

### Logistic Regression

In [9]:
cross_validation_results(logreg)

,fit_time_epl,score_time_epl,test_accuracy_epl,test_precision_epl,test_recall_epl,test_f1_score_epl,test_auc_epl,fit_time_nba,score_time_nba,test_accuracy_nba,...,test_recall_nba,test_f1_score_nba,test_auc_nba,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1_score,test_auc
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.358228,0.005688,0.658850,0.644656,0.526279,0.579295,0.646035,0.271373,0.006224,0.643054,...,0.767295,0.707816,0.623475,0.307100,0.006845,0.600460,0.599902,0.615472,0.607371,0.600384
std,0.022431,0.000588,0.005045,0.009644,0.016055,0.008556,0.004948,0.036122,0.000431,0.019882,...,0.083504,0.033705,0.015938,0.052498,0.000448,0.014009,0.016312,0.019699,0.012713,0.014035
min,0.323655,0.005205,0.652655,0.634731,0.500000,0.567416,0.640376,0.213220,0.005910,0.621943,...,0.635623,0.656250,0.604902,0.235762,0.006252,0.585731,0.584071,0.589450,0.594595,0.585662
25%,0.353080,0.005321,0.654867,0.635294,0.524752,0.574526,0.643327,0.272202,0.005983,0.630535,...,0.732247,0.692350,0.614594,0.274780,0.006669,0.589183,0.586813,0.605505,0.599327,0.589103
50%,0.364719,0.005477,0.659292,0.644970,0.529703,0.580645,0.644000,0.273079,0.005986,0.635403,...,0.818605,0.720930,0.619799,0.313656,0.006821,0.598389,0.594017,0.612385,0.602579,0.598170
75%,0.364928,0.005768,0.663717,0.652439,0.534653,0.584699,0.650851,0.287200,0.006293,0.657860,...,0.820930,0.731606,0.632178,0.347500,0.007010,0.609896,0.616307,0.633867,0.614365,0.609967
max,0.384757,0.006669,0.663717,0.655844,0.542289,0.589189,0.651622,0.311164,0.006948,0.669531,...,0.829070,0.737945,0.645903,0.363803,0.007471,0.619102,0.618304,0.636156,0.625989,0.619017


### K Nearest Neighbours

In [10]:
cross_validation_results(knn)

,fit_time_epl,score_time_epl,test_accuracy_epl,test_precision_epl,test_recall_epl,test_f1_score_epl,test_auc_epl,fit_time_nba,score_time_nba,test_accuracy_nba,...,test_recall_nba,test_f1_score_nba,test_auc_nba,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1_score,test_auc
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.003381,0.046098,0.600885,0.557444,0.507394,0.530911,0.591820,0.004184,0.288210,0.592042,...,0.676096,0.651950,0.578795,0.003431,0.122701,0.551899,0.551815,0.571939,0.561543,0.551805
std,0.000701,0.008462,0.029912,0.034503,0.053635,0.043789,0.031912,0.000874,0.024026,0.023216,...,0.063819,0.034086,0.018552,0.000473,0.014489,0.014317,0.013506,0.029179,0.019791,0.014271
min,0.002603,0.040876,0.555310,0.502762,0.450495,0.475196,0.545248,0.003130,0.266882,0.564442,...,0.592549,0.611779,0.555475,0.002862,0.112934,0.530495,0.531963,0.534404,0.533181,0.530481
25%,0.003028,0.040996,0.592920,0.549708,0.467662,0.505376,0.580444,0.003647,0.269020,0.573034,...,0.621653,0.618413,0.569975,0.003093,0.113976,0.548907,0.550691,0.548165,0.549425,0.548909
50%,0.003271,0.042435,0.603982,0.564246,0.500000,0.530184,0.594000,0.004245,0.281618,0.595112,...,0.722093,0.669542,0.575114,0.003397,0.117521,0.552359,0.551724,0.585812,0.568257,0.552165
75%,0.003524,0.045282,0.617257,0.577540,0.534653,0.555270,0.609327,0.004456,0.298870,0.607662,...,0.722093,0.676471,0.589640,0.003813,0.121098,0.558113,0.554852,0.588101,0.578022,0.557957
max,0.004477,0.060899,0.634956,0.592965,0.584158,0.588529,0.630079,0.005440,0.324662,0.619960,...,0.722093,0.683544,0.603772,0.003988,0.147975,0.569620,0.569845,0.603211,0.578829,0.569513


### Random Forest Classifier

In [11]:
cross_validation_results(forest)

,fit_time_epl,score_time_epl,test_accuracy_epl,test_precision_epl,test_recall_epl,test_f1_score_epl,test_auc_epl,fit_time_nba,score_time_nba,test_accuracy_nba,...,test_recall_nba,test_f1_score_nba,test_auc_nba,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1_score,test_auc
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.488829,0.029645,0.634071,0.614486,0.485577,0.541845,0.619681,1.968386,0.054828,0.633671,...,0.747757,0.697357,0.615694,1.053712,0.034305,0.603452,0.606245,0.602644,0.604188,0.603433
std,0.013749,0.001092,0.019134,0.028899,0.038237,0.029643,0.019803,0.080549,0.005456,0.022757,...,0.072837,0.033922,0.018082,0.006814,0.000592,0.015999,0.018834,0.018244,0.012497,0.016073
min,0.469282,0.028313,0.610619,0.579268,0.432836,0.502890,0.597149,1.836102,0.049051,0.600793,...,0.628638,0.641330,0.596429,1.045475,0.033633,0.581128,0.579176,0.575688,0.587135,0.580955
25%,0.485249,0.029074,0.619469,0.600000,0.470297,0.519126,0.600880,1.958698,0.050551,0.624174,...,0.726426,0.692564,0.597399,1.048320,0.033817,0.593786,0.599045,0.596330,0.594655,0.593849
50%,0.485666,0.029413,0.634956,0.604520,0.480198,0.551136,0.624851,1.999169,0.053448,0.633840,...,0.794186,0.705943,0.619329,1.054093,0.034272,0.606444,0.605322,0.605505,0.611046,0.606338
75%,0.500300,0.030304,0.650442,0.641975,0.514851,0.564644,0.634099,1.999170,0.060271,0.652576,...,0.794186,0.722281,0.630090,1.059664,0.034882,0.616801,0.621176,0.610984,0.613240,0.616840
max,0.503649,0.031123,0.654867,0.646667,0.529703,0.571429,0.641426,2.048789,0.060819,0.656973,...,0.795349,0.724668,0.635225,1.061008,0.034920,0.619102,0.626506,0.624714,0.614865,0.619181


### Support Vector Machine

In [12]:
cross_validation_results(svm)

,fit_time_epl,score_time_epl,test_accuracy_epl,test_precision_epl,test_recall_epl,test_f1_score_epl,test_auc_epl,fit_time_nba,score_time_nba,test_accuracy_nba,...,test_recall_nba,test_f1_score_nba,test_auc_nba,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1_score,test_auc
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.105122,0.012445,0.661504,0.650867,0.522314,0.579375,0.648053,0.282078,0.014378,0.640411,...,0.771948,0.707519,0.619683,0.323477,0.015600,0.602532,0.602689,0.613181,0.607703,0.602471
std,0.014054,0.001564,0.003832,0.009702,0.014051,0.006737,0.003614,0.057352,0.000520,0.022498,...,0.084068,0.035408,0.017507,0.019082,0.001690,0.011085,0.013391,0.018742,0.010894,0.011104
min,0.096166,0.011295,0.654867,0.640244,0.500000,0.570621,0.641901,0.210093,0.013623,0.614673,...,0.640279,0.653595,0.600315,0.299095,0.013480,0.589183,0.588764,0.594037,0.594779,0.589142
25%,0.096773,0.011458,0.661504,0.642857,0.519802,0.573770,0.648000,0.259173,0.014113,0.627892,...,0.734575,0.691507,0.610660,0.309764,0.014883,0.593786,0.593258,0.600917,0.599319,0.593745
50%,0.099834,0.011943,0.663717,0.652439,0.524752,0.582418,0.649135,0.281259,0.014552,0.631440,...,0.825581,0.718750,0.611171,0.327202,0.015182,0.602992,0.598712,0.605505,0.607268,0.602787
75%,0.103060,0.012395,0.663717,0.654321,0.529703,0.584699,0.650376,0.292053,0.014637,0.661823,...,0.829070,0.736082,0.635300,0.334772,0.016537,0.612198,0.611607,0.627002,0.617940,0.612112
max,0.129779,0.015135,0.663717,0.664474,0.537313,0.585366,0.650851,0.367811,0.014966,0.666226,...,0.830233,0.737662,0.640968,0.346553,0.017917,0.614499,0.621103,0.638444,0.619209,0.614570


# Betting Odds Analysis

In [13]:
def run_model(model, sport):
    
    if sport == 'epl':
        x_train, x_test, y_train, y_test = train_test_split(epl_x, epl_y, test_size = 0.2, shuffle=False)
    elif sport == 'nba':
        x_train, x_test, y_train, y_test = train_test_split(nba_x, nba_y, test_size = 0.2, shuffle=False)
    elif sport == 'ufc':
        x_train, x_test, y_train, y_test = train_test_split(ufc_x, ufc_y, test_size = 0.2, shuffle=False)
    else:
        pass
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    y_pred_prob = model.predict_proba(x_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    
    prediction_values_a = []
    prediction_values_b = []
    
    for i in y_pred_prob:
        prediction_values_a.append(i[0])
        prediction_values_b.append(i[1])

    return accuracy, precision, recall, f1, auc, y_pred, y_test, prediction_values_a, prediction_values_b

In [14]:
def show_stats(results):
    
    print("Accuracy:", results[0])
    print("Precision:", results[1])
    print("Recall", results[2])
    print("F1 Score:", results[3])
    print("AUC:", results[4])

In [15]:
def calc_basic_returns(row):
    
    if row['predicted'] == row['actual']:
        if row['predicted'] == 1:
            return (100 * row['A Win'])
        else:
            return (100 * row['B Win'])
    else:
        return 0

In [16]:
def calc_returns_70(row):
    
    if row['A Win Prob'] > 0.70:
        if row['predicted'] == row['actual']:
            return (100 * row['A Win'])
        else:
            return 0
    elif row['B Win Prob'] > 0.70:
        if row['predicted'] == row['actual']:
            return (100 * row['B Win'])
        else:
            return 0
    else:
        return np.nan

In [17]:
def underdog_only(row):
    
    if row['predicted'] == 1:
        if row['A Win'] > row['B Win']:
            if row['predicted'] == row['actual']:
                return (100 * row['A Win'])
            else:
                return 0
        else:
            return np.nan
    else:
        if row['B Win'] > row['A Win']:
            if row['predicted'] == row['actual']:
                return (100 * row['B Win'])
            else:
                return 0
        else:
            return np.nan

In [93]:
def unit_strategy(row):
    
    if row['predicted'] == row['actual']:
        if row['predicted'] == 1:
            if row['A Win Prob'] > 0.7:
                return 200 * row['A Win']
            elif row['A Win Prob'] > 0.6:
                return 100 * row['A Win']
            else:
                return 50 * row['A Win']
        else:
            if row['B Win Prob'] > 0.7:
                return 200 * row['B Win']
            elif row['B Win Prob'] > 0.6:
                return 100 * row['B Win']
            else:
                return 50 * row['B Win']
    else:
        return 0

In [97]:
def unit_bet_calc(row):
    
    if row['A Win Prob'] > 0.7 or row['B Win Prob'] > 0.7:
        return 200
    elif row['A Win Prob'] > 0.6 or row['B Win Prob'] > 0.6:
        return 100
    else:
        return 50

In [98]:
def predictions_dataframe(results, odds):
    
    df = pd.DataFrame(data={'predicted': results[5], 'actual': results[6], 'A Win Prob': results[8], 'B Win Prob': results[7]})
    
    df = df.join(odds)
    
    df['returns'] = df.apply (lambda row: calc_basic_returns(row), axis=1)
    df['returns_70pct'] = df.apply (lambda row: calc_returns_70(row), axis=1)
    df['returns_dogs'] = df.apply (lambda row: underdog_only(row), axis=1)
    df['returns_units'] = df.apply (lambda row: unit_strategy(row), axis=1)
    df['bets_units'] = df.apply (lambda row: unit_bet_calc(row), axis=1)
    
    return df

In [99]:
def betting_summary(df):
    
    base_returns = df['returns'].sum()
    total_bet_base = len(df.index) * 100
    profit_base = base_returns - total_bet_base
    
    returns_70 = df['returns_70pct'].sum()
    total_bets_70 = (len(df.index) - df['returns_70pct'].isnull().sum()) * 100
    profit_70 = returns_70 - total_bets_70
    
    returns_dogs = df['returns_dogs'].sum()
    total_bets_dogs = (len(df.index) - df['returns_dogs'].isnull().sum()) * 100
    profit_dogs = returns_dogs - total_bets_dogs
    
    returns_units = df['returns_units'].sum()
    total_bets_units = df['bets_units'].sum()
    profit_units = returns_units - total_bets_units
    
    print('Basic Strategy - Returns:', base_returns, 'Total Bet:', total_bet_base, 'ROI%:', (profit_base / total_bet_base) * 100) 
    print('70pct Strategy - Returns:', returns_70, 'Total Bet:', total_bets_70, 'ROI%:', (profit_70 / total_bets_70) * 100)
    print('Dogs Strategy - Returns:', returns_dogs, 'Total Bet:', total_bets_dogs, 'ROI%', (profit_dogs / total_bets_dogs) * 100)
    print('Units Stategy - Returns:', returns_units, 'Total Bet:', total_bets_units, 'ROI%', (profit_units / total_bets_units) * 100)

## EPL

### Logistic Regression

In [20]:
regression_epl = run_model(logreg, 'epl')

In [21]:
show_stats(regression_epl)

Accuracy: 0.661504424778761
Precision: 0.5739644970414202
Recall 0.5449438202247191
F1 Score: 0.5590778097982708
AUC: 0.641085048798491


In [100]:
epl_predictions = predictions_dataframe(regression_epl, epl_odds)

In [101]:
epl_predictions

,predicted,actual,A Win Prob,B Win Prob,A Win,B Win,returns,returns_70pct,returns_dogs,returns_units,bets_units
1808,1,1,0.622966,0.377034,1.58,2.43,158.0,NaN,NaN,158.0,100
1809,0,1,0.357557,0.642443,3.02,1.40,0.0,NaN,NaN,0.0,100
1810,0,1,0.276426,0.723574,4.14,1.25,0.0,0.0,NaN,0.0,200
1811,0,0,0.314796,0.685204,2.64,1.50,150.0,NaN,NaN,150.0,100
1812,0,0,0.307017,0.692983,4.79,1.20,120.0,NaN,NaN,120.0,100
...,...,...,...,...,...,...,...,...,...,...,...
2255,1,1,0.687771,0.312229,1.16,5.61,116.0,NaN,NaN,116.0,100
2256,1,1,0.873747,0.126253,1.44,2.87,144.0,144.0,NaN,288.0,200
2257,0,1,0.335634,0.664366,2.84,1.45,0.0,NaN,NaN,0.0,100
2258,0,1,0.428311,0.571689,1.72,2.17,0.0,NaN,0.0,0.0,50


In [102]:
betting_summary(epl_predictions)

Basic Strategy - Returns: 44172.0 Total Bet: 45200 ROI%: -2.274336283185841
70pct Strategy - Returns: 15167.0 Total Bet: 15200 ROI%: -0.21710526315789472
Dogs Strategy - Returns: 6573.0 Total Bet: 7600 ROI% -13.51315789473684
Units Stategy - Returns: 51819.0 Total Bet: 52600 ROI% -1.484790874524715


### K Nearest Neighbours

In [25]:
knn_epl = run_model(knn, 'epl')

In [26]:
show_stats(knn_epl)

Accuracy: 0.6039823008849557
Precision: 0.49710982658959535
Recall 0.48314606741573035
F1 Score: 0.4900284900284901
AUC: 0.5828139096202739


In [103]:
epl_predictions = predictions_dataframe(knn_epl, epl_odds)

In [104]:
epl_predictions

,predicted,actual,A Win Prob,B Win Prob,A Win,B Win,returns,returns_70pct,returns_dogs,returns_units,bets_units
1808,1,1,0.6,0.4,1.58,2.43,158.0,NaN,NaN,79.0,50
1809,0,1,0.2,0.8,3.02,1.40,0.0,0.0,NaN,0.0,200
1810,0,1,0.0,1.0,4.14,1.25,0.0,0.0,NaN,0.0,200
1811,0,0,0.2,0.8,2.64,1.50,150.0,150.0,NaN,300.0,200
1812,0,0,0.2,0.8,4.79,1.20,120.0,120.0,NaN,240.0,200
...,...,...,...,...,...,...,...,...,...,...,...
2255,1,1,0.8,0.2,1.16,5.61,116.0,116.0,NaN,232.0,200
2256,1,1,1.0,0.0,1.44,2.87,144.0,144.0,NaN,288.0,200
2257,1,1,0.8,0.2,2.84,1.45,284.0,284.0,284.0,568.0,200
2258,0,1,0.4,0.6,1.72,2.17,0.0,NaN,0.0,0.0,50


In [105]:
betting_summary(epl_predictions)

Basic Strategy - Returns: 43494.0 Total Bet: 45200 ROI%: -3.7743362831858405
70pct Strategy - Returns: 22736.0 Total Bet: 24600 ROI%: -7.5772357723577235
Dogs Strategy - Returns: 11783.0 Total Bet: 13200 ROI% -10.734848484848484
Units Stategy - Returns: 55851.0 Total Bet: 59500 ROI% -6.132773109243698


### Random Forest

In [30]:
forest_epl = run_model(forest, 'epl')

In [31]:
show_stats(forest_epl)

Accuracy: 0.6194690265486725
Precision: 0.5194805194805194
Recall 0.449438202247191
F1 Score: 0.48192771084337344
AUC: 0.5896826047732305


In [106]:
epl_predictions = predictions_dataframe(forest_epl, epl_odds)

In [107]:
epl_predictions

,predicted,actual,A Win Prob,B Win Prob,A Win,B Win,returns,returns_70pct,returns_dogs,returns_units,bets_units
1808,0,1,0.48,0.52,1.58,2.43,0.0,NaN,0.0,0.0,50
1809,0,1,0.39,0.61,3.02,1.40,0.0,NaN,NaN,0.0,100
1810,0,1,0.33,0.67,4.14,1.25,0.0,NaN,NaN,0.0,100
1811,0,0,0.24,0.76,2.64,1.50,150.0,150.0,NaN,300.0,200
1812,0,0,0.38,0.62,4.79,1.20,120.0,NaN,NaN,120.0,100
...,...,...,...,...,...,...,...,...,...,...,...
2255,1,1,0.70,0.30,1.16,5.61,116.0,NaN,NaN,116.0,100
2256,1,1,0.76,0.24,1.44,2.87,144.0,144.0,NaN,288.0,200
2257,0,1,0.26,0.74,2.84,1.45,0.0,0.0,NaN,0.0,200
2258,0,1,0.37,0.63,1.72,2.17,0.0,NaN,0.0,0.0,100


In [108]:
betting_summary(epl_predictions)

Basic Strategy - Returns: 42151.0 Total Bet: 45200 ROI%: -6.745575221238938
70pct Strategy - Returns: 13519.0 Total Bet: 14700 ROI%: -8.034013605442176
Dogs Strategy - Returns: 7571.0 Total Bet: 9700 ROI% -21.948453608247423
Units Stategy - Returns: 47820.0 Total Bet: 51400 ROI% -6.964980544747082


### Support Vector Machine

In [35]:
svm_epl = run_model(svm, 'epl')

In [36]:
show_stats(svm_epl)

Accuracy: 0.661504424778761
Precision: 0.5748502994011976
Recall 0.5393258426966292
F1 Score: 0.5565217391304347
AUC: 0.6401008775526942


In [109]:
epl_predictions = predictions_dataframe(svm_epl, epl_odds)

In [110]:
epl_predictions

,predicted,actual,A Win Prob,B Win Prob,A Win,B Win,returns,returns_70pct,returns_dogs,returns_units,bets_units
1808,1,1,0.609301,0.390699,1.58,2.43,158.0,NaN,NaN,158.0,100
1809,0,1,0.370765,0.629235,3.02,1.40,0.0,NaN,NaN,0.0,100
1810,0,1,0.303041,0.696959,4.14,1.25,0.0,NaN,NaN,0.0,100
1811,0,0,0.333566,0.666434,2.64,1.50,150.0,NaN,NaN,150.0,100
1812,0,0,0.322667,0.677333,4.79,1.20,120.0,NaN,NaN,120.0,100
...,...,...,...,...,...,...,...,...,...,...,...
2255,1,1,0.653982,0.346018,1.16,5.61,116.0,NaN,NaN,116.0,100
2256,1,1,0.855998,0.144002,1.44,2.87,144.0,144.0,NaN,288.0,200
2257,0,1,0.362158,0.637842,2.84,1.45,0.0,NaN,NaN,0.0,100
2258,0,1,0.440782,0.559218,1.72,2.17,0.0,NaN,0.0,0.0,50


In [111]:
betting_summary(epl_predictions)

Basic Strategy - Returns: 44253.0 Total Bet: 45200 ROI%: -2.095132743362832
70pct Strategy - Returns: 13777.0 Total Bet: 13800 ROI%: -0.16666666666666669
Dogs Strategy - Returns: 6663.0 Total Bet: 7600 ROI% -12.328947368421053
Units Stategy - Returns: 49539.0 Total Bet: 50500 ROI% -1.9029702970297029


## NBA

### Logistic Regression

In [40]:
regression_nba = run_model(logreg, 'nba')

In [41]:
show_stats(regression_nba)

Accuracy: 0.6347424042272127
Precision: 0.6504653567735263
Recall 0.745260663507109
F1 Score: 0.6946438431805633
AUC: 0.6203915257834052


In [112]:
nba_predictions = predictions_dataframe(regression_nba, nba_odds)

In [113]:
nba_predictions

,predicted,actual,A Win Prob,B Win Prob,A Win,B Win,returns,returns_70pct,returns_dogs,returns_units,bets_units
6053,1,1,0.750823,0.249177,1.13,6.25,113.0,113.0,NaN,226.0,200
6054,0,0,0.272484,0.727516,7.25,1.11,111.0,111.0,NaN,222.0,200
6055,0,0,0.260633,0.739367,7.00,1.11,111.0,111.0,NaN,222.0,200
6056,1,1,0.562712,0.437288,1.57,2.55,157.0,NaN,NaN,78.5,50
6057,0,0,0.389793,0.610207,2.40,1.63,163.0,NaN,NaN,163.0,100
...,...,...,...,...,...,...,...,...,...,...,...
7562,0,1,0.471049,0.528951,1.80,2.05,0.0,NaN,0.0,0.0,50
7563,1,0,0.639113,0.360887,2.45,1.61,0.0,NaN,0.0,0.0,100
7564,1,1,0.785992,0.214008,1.36,3.40,136.0,136.0,NaN,272.0,200
7565,1,0,0.650428,0.349572,1.25,4.30,0.0,NaN,NaN,0.0,100


In [114]:
betting_summary(nba_predictions)

Basic Strategy - Returns: 146194.0 Total Bet: 151400 ROI%: -3.438573315719947
70pct Strategy - Returns: 31453.0 Total Bet: 32600 ROI%: -3.51840490797546
Dogs Strategy - Returns: 26985.0 Total Bet: 29800 ROI% -9.446308724832214
Units Stategy - Returns: 145392.0 Total Bet: 151400 ROI% -3.968295904887715


### K Nearest Neighbours

In [45]:
knn_nba = run_model(knn, 'nba')

In [46]:
show_stats(knn_nba)

Accuracy: 0.5653896961690885
Precision: 0.6071428571428571
Recall 0.6244075829383886
F1 Score: 0.6156542056074766
AUC: 0.5577261795288958


In [115]:
nba_predictions = predictions_dataframe(knn_nba, nba_odds)

In [116]:
nba_predictions

,predicted,actual,A Win Prob,B Win Prob,A Win,B Win,returns,returns_70pct,returns_dogs,returns_units,bets_units
6053,1,1,0.6,0.4,1.13,6.25,113.0,NaN,NaN,56.5,50
6054,0,0,0.0,1.0,7.25,1.11,111.0,111.0,NaN,222.0,200
6055,0,0,0.2,0.8,7.00,1.11,111.0,111.0,NaN,222.0,200
6056,0,1,0.4,0.6,1.57,2.55,0.0,NaN,0.0,0.0,50
6057,0,0,0.2,0.8,2.40,1.63,163.0,163.0,NaN,326.0,200
...,...,...,...,...,...,...,...,...,...,...,...
7562,0,1,0.4,0.6,1.80,2.05,0.0,NaN,0.0,0.0,50
7563,1,0,0.6,0.4,2.45,1.61,0.0,NaN,0.0,0.0,50
7564,1,1,0.6,0.4,1.36,3.40,136.0,NaN,NaN,68.0,50
7565,1,0,0.6,0.4,1.25,4.30,0.0,NaN,NaN,0.0,50


In [117]:
betting_summary(nba_predictions)

Basic Strategy - Returns: 142156.0 Total Bet: 151400 ROI%: -6.105680317040951
70pct Strategy - Returns: 73065.0 Total Bet: 76700 ROI%: -4.739243807040417
Dogs Strategy - Returns: 43975.0 Total Bet: 48900 ROI% -10.071574642126789
Units Stategy - Returns: 180675.5 Total Bet: 190750 ROI% -5.281520314547838


### Random Forest

In [50]:
forest_nba = run_model(forest, 'nba')

In [51]:
show_stats(forest_nba)

Accuracy: 0.631439894319683
Precision: 0.649581589958159
Recall 0.735781990521327
F1 Score: 0.69
AUC: 0.6178909952606635


In [118]:
nba_predictions = predictions_dataframe(forest_nba, nba_odds)

In [119]:
nba_predictions

,predicted,actual,A Win Prob,B Win Prob,A Win,B Win,returns,returns_70pct,returns_dogs,returns_units,bets_units
6053,1,1,0.79,0.21,1.13,6.25,113.0,113.0,NaN,226.0,200
6054,0,0,0.28,0.72,7.25,1.11,111.0,111.0,NaN,222.0,200
6055,0,0,0.37,0.63,7.00,1.11,111.0,NaN,NaN,111.0,100
6056,1,1,0.61,0.39,1.57,2.55,157.0,NaN,NaN,157.0,100
6057,0,0,0.39,0.61,2.40,1.63,163.0,NaN,NaN,163.0,100
...,...,...,...,...,...,...,...,...,...,...,...
7562,1,1,0.51,0.49,1.80,2.05,180.0,NaN,NaN,90.0,50
7563,1,0,0.67,0.33,2.45,1.61,0.0,NaN,0.0,0.0,100
7564,1,1,0.77,0.23,1.36,3.40,136.0,136.0,NaN,272.0,200
7565,1,0,0.67,0.33,1.25,4.30,0.0,NaN,NaN,0.0,100


In [120]:
betting_summary(nba_predictions)

Basic Strategy - Returns: 147879.0 Total Bet: 151400 ROI%: -2.3256274768824308
70pct Strategy - Returns: 22302.0 Total Bet: 24700 ROI%: -9.708502024291498
Dogs Strategy - Returns: 31140.0 Total Bet: 33200 ROI% -6.204819277108434
Units Stategy - Returns: 131866.5 Total Bet: 138000 ROI% -4.444565217391305


### Support Vector Machine

In [55]:
svm_nba = run_model(svm, 'nba')

In [56]:
show_stats(svm_nba)

Accuracy: 0.6294583883751651
Precision: 0.6460268317853457
Recall 0.7417061611374408
F1 Score: 0.6905681191395477
AUC: 0.6148829313149891


In [121]:
nba_predictions = predictions_dataframe(svm_nba, nba_odds)

In [122]:
nba_predictions

,predicted,actual,A Win Prob,B Win Prob,A Win,B Win,returns,returns_70pct,returns_dogs,returns_units,bets_units
6053,1,1,0.751754,0.248246,1.13,6.25,113.0,113.0,NaN,226.0,200
6054,0,0,0.285616,0.714384,7.25,1.11,111.0,111.0,NaN,222.0,200
6055,0,0,0.268287,0.731713,7.00,1.11,111.0,111.0,NaN,222.0,200
6056,1,1,0.563744,0.436256,1.57,2.55,157.0,NaN,NaN,78.5,50
6057,0,0,0.412787,0.587213,2.40,1.63,163.0,NaN,NaN,81.5,50
...,...,...,...,...,...,...,...,...,...,...,...
7562,0,1,0.487091,0.512909,1.80,2.05,0.0,NaN,0.0,0.0,50
7563,1,0,0.641806,0.358194,2.45,1.61,0.0,NaN,0.0,0.0,100
7564,1,1,0.785098,0.214902,1.36,3.40,136.0,136.0,NaN,272.0,200
7565,1,0,0.653443,0.346557,1.25,4.30,0.0,NaN,NaN,0.0,100


In [123]:
betting_summary(nba_predictions)

Basic Strategy - Returns: 144892.0 Total Bet: 151400 ROI%: -4.298546895640687
70pct Strategy - Returns: 30772.0 Total Bet: 32100 ROI%: -4.137071651090342
Dogs Strategy - Returns: 26940.0 Total Bet: 30500 ROI% -11.672131147540982
Units Stategy - Returns: 144163.0 Total Bet: 150850 ROI% -4.432880344713292


## UFC

### Logistic Regression

In [60]:
regression_ufc = run_model(logreg, 'ufc')

In [61]:
show_stats(regression_ufc)

Accuracy: 0.5972382048331415
Precision: 0.6008676789587852
Recall 0.6252821670428894
F1 Score: 0.6128318584070797
AUC: 0.5966786422068907


In [124]:
ufc_predictions = predictions_dataframe(regression_ufc, ufc_odds)

In [125]:
ufc_predictions

,predicted,actual,A Win Prob,B Win Prob,A Win,B Win,returns,returns_70pct,returns_dogs,returns_units,bets_units
3476,0,1,0.495404,0.504596,1.34,3.45,0.0,NaN,0.0,0.0,50
3477,1,1,0.675264,0.324736,1.59,2.50,159.0,NaN,NaN,159.0,100
3478,1,0,0.617143,0.382857,1.42,3.00,0.0,NaN,NaN,0.0,100
3479,0,1,0.433274,0.566726,3.50,1.33,0.0,NaN,NaN,0.0,50
3480,1,1,0.631341,0.368659,1.43,2.90,143.0,NaN,NaN,143.0,100
...,...,...,...,...,...,...,...,...,...,...,...
4340,0,0,0.432686,0.567314,1.87,1.95,195.0,NaN,195.0,97.5,50
4341,0,0,0.454921,0.545079,3.43,1.35,135.0,NaN,NaN,67.5,50
4342,1,1,0.519260,0.480740,1.53,2.65,153.0,NaN,NaN,76.5,50
4343,0,0,0.248642,0.751358,1.61,2.45,245.0,245.0,245.0,490.0,200


In [126]:
betting_summary(ufc_predictions)

Basic Strategy - Returns: 91541.0 Total Bet: 86900 ROI%: 5.340621403912543
70pct Strategy - Returns: 22978.0 Total Bet: 17800 ROI%: 29.089887640449437
Dogs Strategy - Returns: 34457.0 Total Bet: 31500 ROI% 9.387301587301588
Units Stategy - Returns: 93404.0 Total Bet: 83200 ROI% 12.264423076923077


### K Nearest Neighbours

In [65]:
knn_ufc = run_model(knn, 'ufc')

In [66]:
show_stats(knn_ufc)

Accuracy: 0.5523590333716916
Precision: 0.5589519650655022
Recall 0.5778781038374717
F1 Score: 0.5682574916759157
AUC: 0.5518498500408016


In [127]:
ufc_predictions = predictions_dataframe(knn_ufc, ufc_odds)

In [128]:
ufc_predictions

,predicted,actual,A Win Prob,B Win Prob,A Win,B Win,returns,returns_70pct,returns_dogs,returns_units,bets_units
3476,1,1,0.6,0.4,1.34,3.45,134.0,NaN,NaN,67.0,50
3477,1,1,0.6,0.4,1.59,2.50,159.0,NaN,NaN,79.5,50
3478,1,0,0.8,0.2,1.42,3.00,0.0,0.0,NaN,0.0,200
3479,0,1,0.2,0.8,3.50,1.33,0.0,0.0,NaN,0.0,200
3480,1,1,0.6,0.4,1.43,2.90,143.0,NaN,NaN,71.5,50
...,...,...,...,...,...,...,...,...,...,...,...
4340,0,0,0.4,0.6,1.87,1.95,195.0,NaN,195.0,97.5,50
4341,0,0,0.4,0.6,3.43,1.35,135.0,NaN,NaN,67.5,50
4342,1,1,0.6,0.4,1.53,2.65,153.0,NaN,NaN,76.5,50
4343,0,0,0.4,0.6,1.61,2.45,245.0,NaN,245.0,122.5,50


In [129]:
betting_summary(ufc_predictions)

Basic Strategy - Returns: 89657.0 Total Bet: 86900 ROI%: 3.172612197928654
70pct Strategy - Returns: 44073.0 Total Bet: 40000 ROI%: 10.1825
Dogs Strategy - Returns: 40888.0 Total Bet: 39400 ROI% 3.7766497461928936
Units Stategy - Returns: 110938.0 Total Bet: 103450 ROI% 7.2382793620106325


### Random Forest

In [70]:
ufc_forest = run_model(forest, 'ufc')

In [71]:
show_stats(ufc_forest)

Accuracy: 0.5914844649021864
Precision: 0.5964912280701754
Recall 0.6139954853273137
F1 Score: 0.6051167964404894
AUC: 0.5910353013491029


In [130]:
ufc_predictions = predictions_dataframe(ufc_forest, ufc_odds)

In [131]:
ufc_predictions

,predicted,actual,A Win Prob,B Win Prob,A Win,B Win,returns,returns_70pct,returns_dogs,returns_units,bets_units
3476,0,1,0.41,0.59,1.34,3.45,0.0,NaN,0.0,0.0,50
3477,1,1,0.66,0.34,1.59,2.50,159.0,NaN,NaN,159.0,100
3478,1,0,0.58,0.42,1.42,3.00,0.0,NaN,NaN,0.0,50
3479,0,1,0.46,0.54,3.50,1.33,0.0,NaN,NaN,0.0,50
3480,1,1,0.73,0.27,1.43,2.90,143.0,143.0,NaN,286.0,200
...,...,...,...,...,...,...,...,...,...,...,...
4340,0,0,0.33,0.67,1.87,1.95,195.0,NaN,195.0,195.0,100
4341,0,0,0.40,0.60,3.43,1.35,135.0,NaN,NaN,67.5,50
4342,1,1,0.54,0.46,1.53,2.65,153.0,NaN,NaN,76.5,50
4343,0,0,0.42,0.58,1.61,2.45,245.0,NaN,245.0,122.5,50


In [132]:
betting_summary(ufc_predictions)

Basic Strategy - Returns: 92321.0 Total Bet: 86900 ROI%: 6.238204833141542
70pct Strategy - Returns: 9376.0 Total Bet: 7600 ROI%: 23.36842105263158
Dogs Strategy - Returns: 37263.0 Total Bet: 33600 ROI% 10.901785714285714
Units Stategy - Returns: 74522.0 Total Bet: 67150 ROI% 10.978406552494416


### Support Vector Machines

In [75]:
svm_ufc = run_model(svm, 'ufc')

In [76]:
show_stats(svm_ufc)

Accuracy: 0.6052934407364787
Precision: 0.6086956521739131
Recall 0.6320541760722348
F1 Score: 0.62015503875969
AUC: 0.6047594824023146


In [133]:
ufc_predictions = predictions_dataframe(svm_ufc, ufc_odds)

In [134]:
betting_summary(ufc_predictions)

Basic Strategy - Returns: 92840.0 Total Bet: 86900 ROI%: 6.8354430379746836
70pct Strategy - Returns: 19628.0 Total Bet: 14300 ROI%: 37.25874125874126
Dogs Strategy - Returns: 34965.0 Total Bet: 31200 ROI% 12.067307692307692
Units Stategy - Returns: 89102.0 Total Bet: 78150 ROI% 14.01407549584133
